# Multiple resonators on a chip, with possibility of having holes in the resonator core

Ruirements: gdspy library.
The design is generated in the following order:


   *   **Ground and contact pads**. THe idea is that you should only specify the coordinates of the contact pads; whether they should be rotated should be decided by the code itself (if the contact pad is further that 900 $\mu m$ away, it's rotated). This is done by the functions 
   
   *Generate_Pads (sample_size_along_x_axis, sample_size_along_y_axis, [d], [d1], [d2], [layer], [contact pad coordinates])*. 
   
   
   *   **TL**. Connects two contact pads from left to right. Generally, in this code everything that involves coplanar waveguides should be drawn from left to right.
   
   *Generate_TL (contact_pad_1, contact_pad2, d, d1, d2)*
   
    
   *   **Resonators** The position is specified by the left lower corner in (x,y) format. 
   
       *Generate_Resonators (initial_point_array, [circular_claw_radius], [gap], [resonator_frequency], [mode (up or down)], [layer])*.
   
   *   **DC line**. At this point the we add resonators and dc bias lines to the sample cell.
   
   *Generate_Loops* ([coordinates of the second end; the first end automatically gets connected to a contact pad], [modes], [d], [d1], [d2], [d for the contact pad], [d1 for the contact pad], [d2 for the contact pad], [array of arc coordinates], [array of arc angles], layer)
   
   
   *   **Ground grid**. Can be generated only after everything else is in place
   
   *Generate_Grid* (sample_size_along_x, sample_size_along_y, layer1, layer2)
   
 IF you want to change anything, unfortunately you'll have to do a restart.

In [1]:
!pip install gdspy

In [2]:
from designTL import *
from resonator_holes_TL import*
import gdspy

In [3]:
class Sample:
    
    #Содержит функции, генерирующие все площадки, TL, все джозефсоны, все петельки
    
    def __init__(self, number):
        self.name = 'sample' + str(number)
        self.cell = gdspy.Cell(self.name)
    
    def Generate_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads_cell = gdspy.Cell('Pads')
        self.Pads_cell.add(self.Pads.Generate_Ground())
        self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
        
    def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
        TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
        self.tl = TL
        self.tl_cell1 = gdspy.Cell('TL1')
        self.tl_cell2 = gdspy.Cell('TL2')
        self.tl_cell3 = gdspy.Cell('TL3')
        self.tl_cell1.add(self.tl.GenerateTL1(coords = [], rots = [])) 
        self.tl_cell2.add(TL.GenerateTL2(coords = [], rots = [])) 
        self.tl_cell3.add(TL.GenerateTL3(coords = [], rots = [])) 
        #self.cell.add([gdspy.CellReference(self.tl_cell, (0, 0))])
      
    def Generate_Resonators(self, start_points, freqs, modes, layer, ws, ss, hds, hdensities, TL_ground):
        self.RESONATORS1 = gdspy.Cell('RESONATORS1')
        self.RESONATORS2 = gdspy.Cell('RESONATORS2')
        self.RESONATORS3 = gdspy.Cell('RESONATORS3')
        self.resonators = []
        self.res_cells = []
        self.res_references = []
        for i, (start, freq, mode, w, s, hd, hdensity, TL) in enumerate(zip(start_points, freqs, modes, ws, ss, hds, hdensities, TL_ground)):
            self.resonators.append(Resonator(start.x, start.y, freq, layer, w, s, hd, hdensity, TL))
            self.res_cells.append(gdspy.Cell('res' + str(i)))
            self.RESONATORS1.add(self.resonators[i].Generate1(mode))
            self.RESONATORS2.add(self.resonators[i].Generate2(mode))
            self.RESONATORS3.add(self.resonators[i].Generate3(mode))
            self.res_cells[i].add(self.resonators[i].Generate3(mode))
            
        COMBINATION = gdspy.Cell('COMBINATION')
        COMBINATION.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS3, (0,0)), gdspy.CellReference(self.tl_cell3, (0,0)), 'or'))
        CUT = gdspy.Cell('CUT')
        CUT.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS2, (0,0)), gdspy.CellReference(self.tl_cell2, (0,0)), 'or'))
        CUTOUT = gdspy.Cell('CUTOUT')
        CUTOUT.add(gdspy.boolean(gdspy.CellReference(COMBINATION, (0,0)), gdspy.CellReference(CUT, (0,0)), 'not'))
        FILL = gdspy.Cell('FILL')
        FILL.add(gdspy.boolean(gdspy.CellReference(self.RESONATORS1, (0,0)), gdspy.CellReference(self.tl_cell1, (0,0)), 'or'))
        
        CUTOUTFILL = gdspy.Cell('CUTOUTFILL')
        CUTOUTFILL.add(gdspy.boolean(gdspy.CellReference(CUTOUT, (0,0)), gdspy.CellReference(FILL, (0,0)), 'or'))
        self.cell.add(gdspy.CellReference(CUTOUTFILL, (0, 0)))

        
                        
    def Generate_Grid(self, a, b, l1, l2):
        self.grid_cell = gdspy.Cell('Grid')
        res1 = gdspy.Rectangle((400, 400), (402, b - 400), layer = l1) 
        x = 447
        while x < a - 400:
            r1 = gdspy.Rectangle((x, 400), (x + 2, b - 400), layer = l1) 
            x += 47
            res1 = gdspy.boolean(res1, r1, 'or')
            #self.grid_cell.add(r1)
        y = 400
        res2 = gdspy.Rectangle((400, 400), (400, 402), layer = l2) 
        while y < b - 400:
            r1 = gdspy.Rectangle((400, y), (a - 400, y + 2), layer = l2) 
            y += 47
            res2 = gdspy.boolean(res2, r1, 'or')
            #self.grid_cell.add(r1)
        res2 = gdspy.boolean(res2, res1, 'not')
        for area in self.Pads.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for area in self.tl.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for i in range(len(self.resonators)):
            area = self.resonators[i].restricted_area
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        
        #res = gdspy.boolean(res, self.Pads_cell.get_polygonsets(), 'not')
        self.grid_h = res1
        self.grid_v = res2
        self.grid_cell.add(res1)
        self.grid_cell.add(res2)
        self.cell.add(gdspy.CellReference(self.grid_cell, (0, 0)))
    #self.Resonators
    #self.JJ
    #self.Loops
    #self.TL

In [4]:
sample = Sample(2)
TL_inner = 14
TL_vacuum = 8
TL_ground = 50
sample.Generate_Pads(10000, 5000, #width, depth
                     2, #number of Pads
                     [TL_inner, TL_inner], #d
                     [TL_inner +2*TL_vacuum, TL_inner +2*TL_vacuum], #d1
                     [TL_inner + 2*(TL_vacuum+TL_ground), TL_inner + 2*(TL_vacuum+TL_ground)], #d2
                     0, #layer
                     [coordinates(800, 2500), coordinates(9200, 2500)]) #reference coordinates for Pads
sample.Generate_TL(0, 1, TL_inner, TL_inner +2*TL_vacuum, TL_inner + 2*(TL_vacuum+TL_ground), 0)
sample.Generate_Resonators([coordinates(sample.tl.start.x + 50 +60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(sample.tl.start.x + 50+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2),
                           coordinates(1422+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(1665+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2),
                           coordinates(2317+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(2529+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2), 
                           coordinates(2907+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(3327+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2), 
                           coordinates(3396+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(4062+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2),
                           coordinates(4054+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(4668+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2),
                           coordinates(4592+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(5160+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2),
                           coordinates(5084+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(5702+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2), 
                           coordinates(5704+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(6416+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2),
                           coordinates(6374+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(7155+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2), 
                           coordinates(7207+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(7629+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2),
                           coordinates(7874+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d),
                           coordinates(8262+60, 
                                        sample.tl.start.y - (sample.tl.d2 - sample.tl.d)/2), 
                           coordinates(8468+60, 
                                        sample.tl.start.y + (sample.tl.d2 - sample.tl.d)/2 + sample.tl.d)], # ref coordinates
                           [9.75e9, 5.25e9, 5.75e9, 5e9, 8.25e9, 8e9, 9.5e9, 10.75e9, 9e9, 7.75e9, 7.25e9, 8.75e9, 10e9, 8.5e9, 5.5e9, 6.5e9, 6e9, 10.5e9, 7e9, 11e9, 6.25e9, 6.75e9, 9.25e9, 10.25e9, 7.5e9], #freqs
                           ['up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up', 'down', 'up'], #mode
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #layers
                           [8.284, 16.569, 16.569, 9.428, 6.667, 20, 2.612, 13.333, 7.388, 11.716, 3.333, 5.858, 3.694, 5.858, 4.142, 5.224, 5, 23.431, 10.448, 7.071, 8.284, 4.714, 10, 14.142, 11.716], #w
                           [2.929, 5.858, 11.716, 9.428, 6.667, 10, 3.694, 13.333, 10.448, 4.142, 3.333, 2.071, 5.224, 4.142, 2.929, 7.338, 2.5, 8.284, 14.776, 3.536, 5.858, 4.714, 5, 7.071, 8.2843], #s
                           [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], #the hole diameter 
                           [0.1, 0.01, 0.01, 0.1, 0, 0, 0, 0, 0, 0.01, 0, 0, 0, 0, 0, 0, 0, 0.1, 0, 0, 0, 0, 0.1, 0.1, 0.01], #the hole density
                           [TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground, TL_ground]) 

coupling length: 202.96476698916402
w= 8.284
s= 2.9289999999999994
w/s= 2.8282690337999323
w+2s= 14.142
x coordinate of end of resonator 1211.9587669891641
DE= 12.285333333333334
...
coupling length: 376.93456726559026
w= 16.569
s= 5.857999999999999
w/s= 2.828439740525777
w+2s= 28.284999999999997
x coordinate of end of resonator 1484.9295672655903
DE= 18.571111111111108
...
coupling length: 344.1576483729302
w= 16.569
s= 11.716
w/s= 1.4142198702628883
w+2s= 40.001
x coordinate of end of resonator 2106.1646483729305
DE= 23.778222222222222
...
coupling length: 395.7812956288697
w= 9.428
s= 9.428
w/s= 1.0
w+2s= 28.284000000000002
x coordinate of end of resonator 2318.7692956288697
DE= 18.570666666666668
...
coupling length: 239.86745189628473
w= 6.667
s= 6.666999999999999
w/s= 1.0000000000000002
w+2s= 20.000999999999998
x coordinate of end of resonator 2756.8744518962844
DE= 14.889333333333333
...
coupling length: 247.3633097680436
w= 20
s= 10.0
w/s= 2.0
w+2s= 40
x coordinate of end of re

In [5]:
#gdspy.LayoutViewer(depth = 2)

In [6]:
sample.Generate_Grid(10000,5000,0,0)

In [7]:
gdspy.write_gds('resonators2.gds', name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)